In [1]:
import time

start = time.time()

In [2]:
import concurrent.futures
import json
import os
import re
import sys
import typing as T
from threading import Lock

import dotenv
import pandas as pd

dotenv.load_dotenv()

# Secrets
GOOGLE_API_KEY = os.getenv("GOOGLE_PLACES_API_KEY")
assert GOOGLE_API_KEY is not None, "GOOGLE_PLACES_API_KEY is not set in the `.env` file"
OPEN_AI_API_KEY = os.getenv("OPEN_AI_API_KEY")
assert OPEN_AI_API_KEY is not None, "OPEN_AI_API_KEY is not set in the `.env` file"
MAPBOX_API_KEY = os.getenv("MAPBOX_API_KEY")
assert MAPBOX_API_KEY is not None, "MAPBOX_API_KEY is not set in the `.env` file"

CURRENT_DIR = %pwd
ROOT_DIR = os.path.dirname(CURRENT_DIR)
SRC_DIR = os.path.join(ROOT_DIR, "src")

sys.path.append(SRC_DIR)

In [3]:
from llm.defs import DAY_COLUMN, ACTIVITY_TYPE_COLUMN, LOCATION_COLUMN, DESCRIPTION_COLUMN
from google.utils import TYPES, DEFAULT_TYPE

UI_INPUT = False

In [4]:
inputs = {
    "location": "South Beach Miami, FL",
    "number_of_people": 6,
    "date": "November 2026",
    "duration_days": 3,
    "group_type": "bachelorette party",
    "description": (
        "include boutique hotel options must 4 stars higher onsite spa beach clubs djs"
        "day high end nightclubs list least six restaurant options dinner include least "
        "one nice steakhouse one nice sushi restaurant"
    ),
}

In [5]:
if UI_INPUT:
    ##############################################################################
    # Skip this block if you want to use the hard coded inputs
    ##############################################################################
    from input_ui import SimpleInputForm

    form = SimpleInputForm()
    form.setup()
    inputs = form.run_form()

In [6]:
from llm.search import OpenAiSearch
from llm.defs import ITINERARY_PROMPT_TEMPLATE, Itinerary

llm = OpenAiSearch(OPEN_AI_API_KEY)
output = llm.search(inputs, ITINERARY_PROMPT_TEMPLATE, Itinerary)

print(output)

input_tokens, output_tokens, total_tokens = llm.calculate_tokens(inputs, output)
print(f"Input Tokens: {input_tokens}")
print(f"Output Tokens: {output_tokens}")
print(f"Total Tokens: {total_tokens}")

/home/ross/Documents/python/upwork/reimagined-funicular/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


{'day': ['Day 1', 'Day 1', 'Day 1', 'Day 1', 'Day 1', 'Day 1', 'Day 2', 'Day 2', 'Day 2', 'Day 2', 'Day 2', 'Day 2', 'Day 3', 'Day 3', 'Day 3', 'Day 3', 'Day 3', 'Day 3'], 'activity_type': ['Breakfast', 'Morning Activity', 'Lunch', 'Afternoon Activity', 'Dinner', 'Evening Activity', 'Breakfast', 'Morning Activity', 'Lunch', 'Afternoon Activity', 'Dinner', 'Evening Activity', 'Breakfast', 'Morning Activity', 'Lunch', 'Afternoon Activity', 'Dinner', 'Evening Activity'], 'location': ['The Local House', 'South Beach', 'Yardbird Southern Table and Bar', 'South Pointe Park', 'Prime 112', 'LIV Nightclub', 'The Front Porch Cafe', 'South Beach Kayak', "Joe's Stone Crab", 'Wynwood Walls', 'Komodo', 'E11EVEN Miami', 'The Social Club', 'South Beach Parasail', 'Big Pink', 'Lincoln Road Mall', 'Nobu Miami', 'Story Nightclub'], 'description': ['Chic Breakfast Spot', 'Beach Relaxation', 'Southern Comfort Food', 'Park with Ocean Views', 'Upscale Steakhouse', 'High-End Nightclub', 'Cozy Cafe with Outdoo

In [7]:
# Extract the itinerary content from the response
df = pd.DataFrame(output)

print(df)

      day       activity_type                         location  \
0   Day 1           Breakfast                  The Local House   
1   Day 1    Morning Activity                      South Beach   
2   Day 1               Lunch  Yardbird Southern Table and Bar   
3   Day 1  Afternoon Activity                South Pointe Park   
4   Day 1              Dinner                        Prime 112   
5   Day 1    Evening Activity                    LIV Nightclub   
6   Day 2           Breakfast             The Front Porch Cafe   
7   Day 2    Morning Activity                South Beach Kayak   
8   Day 2               Lunch                 Joe's Stone Crab   
9   Day 2  Afternoon Activity                    Wynwood Walls   
10  Day 2              Dinner                           Komodo   
11  Day 2    Evening Activity                    E11EVEN Miami   
12  Day 3           Breakfast                  The Social Club   
13  Day 3    Morning Activity             South Beach Parasail   
14  Day 3 

In [12]:
# Inputs for the nearby lookup, if store_type is left as None, it will
# default to match the type of the place that we are searching nearby from
MAX_NEARBY_PLACES = 2

keyword = "breakfast"
store_type = None  # None will default to the type of the place that we are searching nearby from
radius_meters = 1500
# radius_from_place_miles = 1.0
# radius_meters = miles_to_meters(radius_from_place_miles)

In [13]:
from google.search import SearchPlaces

search = SearchPlaces(GOOGLE_API_KEY, verbose=True)
itinerary_place_details, nearby_place_details, city_coordinates = search.search(
    city=inputs["location"],
    itinerary=df,
    store_type=store_type,
    keyword=keyword,
    radius_meters=radius_meters,
)
print(f"Num results: {len(itinerary_place_details)}")
for place, item in itinerary_place_details:
    print(item["name"])
    nearby_names = [i["name"] for i in nearby_place_details.get(place, [])]
    for name in nearby_names:
        print(f"  - {name}")

South Beach Miami, FL coordinates: (25.7744291, -80.1332415)
Getting nearby places for The Local House at {'lat': 25.773442, 'lng': -80.132473}
Found 20 nearby places for The Local House
Getting nearby places for South Beach at {'lat': 25.7826123, 'lng': -80.1340772}
Unable to get nearby places info for South Beach
Getting nearby places for Yardbird Southern Table and Bar at {'lat': 25.7890685, 'lng': -80.14015789999999}
Found 20 nearby places for Yardbird Southern Table and Bar
Getting nearby places for South Pointe Park at {'lat': 25.7663809, 'lng': -80.134671}
Unable to get nearby places info for South Pointe Park
Getting nearby places for Prime 112 at {'lat': 25.76992, 'lng': -80.13326699999999}
Found 20 nearby places for Prime 112
Getting nearby places for LIV Nightclub at {'lat': 25.8178372, 'lng': -80.1227119}
Found 3 nearby places for LIV Nightclub
Getting nearby places for The Front Porch Cafe at {'lat': 25.7871279, 'lng': -80.1297932}
Found 20 nearby places for The Front Porc

In [10]:
total_time = time.time() - start

print(f"Total time taken: {total_time:.2f} seconds")

Total time taken: 23.84 seconds


In [11]:
import plotly.express as px
import plotly.graph_objects as go

assert len(itinerary_place_details) != 0, "No places found in the itinerary"
assert len(nearby_place_details) != 0, "No nearby places found"

# Create a map showing the places in the itinerary and nearby places
print("Creating map...")
px.set_mapbox_access_token(MAPBOX_API_KEY)

places_dfs = {}
color = 0
color_increment = 1.0 / len(itinerary_place_details)
for name, place in itinerary_place_details:
    color += color_increment
    rows = [(place["name"], *place["geometry"]["location"].values(), color)]
    max_clamp = min(MAX_NEARBY_PLACES, len(nearby_place_details.get(name, [])))
    rows.extend(
        [
            (x["name"], *x["geometry"]["location"].values(), color)
            for x in nearby_place_details.get(name, [])[:max_clamp]
        ]
    )
    places_dfs[name] = pd.DataFrame(rows, columns=["name", "latitude", "longitude", "color"])


fig = go.Figure()

for row in df.iterrows():
    places_coords_df = places_dfs[row[1][LOCATION_COLUMN]]
    trace_name = (
        f"Day {row[1][DAY_COLUMN]}: {row[1][ACTIVITY_TYPE_COLUMN]} at {row[1][LOCATION_COLUMN]}"
    )
    fig.add_trace(
        go.Scattermapbox(
            lat=places_coords_df["latitude"],
            lon=places_coords_df["longitude"],
            mode="markers",
            marker=go.scattermapbox.Marker(
                size=5, color=places_coords_df["color"], colorscale="Rainbow", cmin=0, cmax=1
            ),
            text=places_coords_df["name"],
            name=trace_name,
        )
    )

fig.update_layout(
    autosize=True,
    hovermode="closest",
    mapbox=go.layout.Mapbox(
        accesstoken=os.getenv("MAPBOX_API_KEY"),
        bearing=0,
        center=go.layout.mapbox.Center(lat=city_coordinates[0], lon=city_coordinates[1]),
        pitch=0,
        zoom=12,
    ),
    height=800,
    width=1000,
    title_text=f"Itinerary for `{inputs['group_type']}` in {inputs['location']}",
    title_x=0.5,
    annotations=[
        dict(
            text=(
                f"Nearby {MAX_NEARBY_PLACES} places for "
                f"`{keyword}` near the places for each itinerary"
            ),
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0.5,
            y=0.0,
            xanchor="center",
            yanchor="top",
            font=dict(size=14, color="black"),
        )
    ],
)

fig.show()

Creating map...
